# Variables 

## 引入需要的包

In [1]:
import numpy as np
import tensorflow as tf
#graph_tensor = tf.Graph()

## 创建变量

可以使用tf.Variable与tf.get_variable两种方式创建变量

### tf.Variable
必须给定的参数只有一个initial_value，如果名字没指定会自己生成一个Variable:0类似于此

In [2]:
"""
__init__(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None,
    use_resource=None,
    synchronization=tf.VariableSynchronization.AUTO,
    aggregation=tf.VariableAggregation.NONE
)
"""

'\n__init__(\n    initial_value=None,\n    trainable=True,\n    collections=None,\n    validate_shape=True,\n    caching_device=None,\n    name=None,\n    variable_def=None,\n    dtype=None,\n    expected_shape=None,\n    import_scope=None,\n    constraint=None,\n    use_resource=None,\n    synchronization=tf.VariableSynchronization.AUTO,\n    aggregation=tf.VariableAggregation.NONE\n)\n'

### get_variable
get_variable：必须给定的参数只有name

In [3]:
"""
def get_variable(name,
                 shape=None,
                 dtype=None,
                 initializer=None,
                 regularizer=None,
                 trainable=True,
                 collections=None,
                 caching_device=None,
                 partitioner=None,
                 validate_shape=True,
                 use_resource=None,
                 custom_getter=None):
"""

'\ndef get_variable(name,\n                 shape=None,\n                 dtype=None,\n                 initializer=None,\n                 regularizer=None,\n                 trainable=True,\n                 collections=None,\n                 caching_device=None,\n                 partitioner=None,\n                 validate_shape=True,\n                 use_resource=None,\n                 custom_getter=None):\n'

tf.Variable是用于生成一个新的变量，tf.get_variable是获取一个已经存在的变量，如果不存在再重建。

在tensorflow中每个变量都有一个scope也就是命名空间，一个变量要的全名由scope和name唯一确定，因为tf.Variable不涉极变量复用问题，命名空间意义不是太明显，

只有在用tf.get_variable时才会体会用命名空间的作用。 

### 通过tf.Variabe 创建变量

In [4]:
import tensorflow as tf;
import numpy as np;

v1 = tf.Variable(initial_value=[[1.0,2.0,3.0], [1.0,2.0,3.0]], name="v1")
v2 = tf.Variable(initial_value=[[1.0,2.0,3.0], [1.0,2.0,3.0]], name="v2")
v3 = tf.Variable(initial_value=[[1.0,2.0,3.0], [1.0,2.0,3.0]], name="v1")

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(v1))
        print(sess.run(v2))
        print(sess.run(v3))
        print(v1)
        print(v2)
        print(v3)

Instructions for updating:
Colocations handled automatically by placer.
[[1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]]
[[1. 2. 3.]
 [1. 2. 3.]]
<tf.Variable 'v1:0' shape=(2, 3) dtype=float32_ref>
<tf.Variable 'v2:0' shape=(2, 3) dtype=float32_ref>
<tf.Variable 'v1_1:0' shape=(2, 3) dtype=float32_ref>


variable_scope是变量的命名空间。

此处没有调用with tf.variable_scope(“scope_1”):，可以认为所有变量的scope为空。

v3定义时name=”v1”,在这种情况下tensorflow会把v3重命名。

也就是tf.Variable无论何时都能创建成功，无论是否有同名变量。

### 通过tf.get_variable 创建变量

下方代码会报错，因为在当前空间中存在同名的变量v1。

In [5]:
with tf.Session() as sess:
    v1 = tf.get_variable(name='v1', shape=[2, 3], initializer=tf.random_normal_initializer(mean=0, stddev=1))
    sess.run(tf.global_variables_initializer())
    print(sess.run(v1))

[[-1.5101554   0.2484174  -1.144254  ]
 [-0.03578377 -0.71004486  1.1033803 ]]


所以我们需要在该变量前加一个scope进行命名空间的声明

In [6]:
with tf.Session() as sess:
    with tf.variable_scope("scope_1", reuse=True):
        v1 = tf.get_variable(name='v1', shape=[2, 3], initializer=tf.random_normal_initializer(mean=0, stddev=1))
        sess.run(tf.global_variables_initializer())
        print(sess.run(v1))

ValueError: Variable scope_1/v1 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?

其中reuse=True表示如果有已经存在的变量则可以直接获取已有的变量。

reuse=tf.AUTO_REUSE表示如果变量已经存在，则加载当前的变量，如果没有则创建。

下方代码如果把reuse设为None或者False，都会报错

In [7]:
with tf.Session() as sess:
    with tf.variable_scope("scope_1", reuse=None):
        v1 = tf.get_variable(name='v1', shape=[2, 3], initializer=tf.random_normal_initializer(mean=0, stddev=1))
        sess.run(tf.global_variables_initializer())
        print(sess.run(v1))

[[ 0.09226608  0.652639   -1.0387292 ]
 [ 0.519874    0.13647395  0.7273705 ]]


In [8]:
with tf.variable_scope('var', reuse=tf.AUTO_REUSE):
    my_variable = tf.get_variable("my_variable", [1, 2, 3])

这将创建一个名为“my_variable”的变量，该变量是形状为 [1, 2, 3] 的三维张量。默认情况下，此变量将具有 dtypetf.float32。

可以选择为 tf.get_variable 指定 dtype 和初始化器。例如：

In [9]:
with tf.variable_scope('var', reuse=tf.AUTO_REUSE):
    my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32, initializer=tf.zeros_initializer)

TensorFlow 提供了许多方便的初始化器。或者，也可以将 tf.constant 初始化为 tf.Tensor 的值。例如：


In [10]:
with tf.variable_scope('var', reuse=tf.AUTO_REUSE):
    other_variable = tf.get_variable("other_variable", dtype=tf.int32, initializer=tf.constant([23, 42]))

请注意，当初始化器是 tf.Tensor 时，不应指定变量的形状，因为将使用初始化器张量的形状。

## 初始化变量
变量必须先初始化后才可使用。如果在低级别 TensorFlow API 中进行编程（即在显式创建自己的图和会话），则必须明确初始化变量。

tf.estimator.Estimator 和 Keras 等大多数高级框架在训练模型前会自动为您初始化变量。

要在训练开始前一次性初始化所有可训练变量，请调用 tf.global_variables_initializer()。

此函数会返回一个操作，负责初始化 tf.GraphKeys.GLOBAL_VARIABLES 集合中的所有变量。运行此操作会初始化所有变量。例如：

In [11]:
session = tf.Session()
session.run(tf.global_variables_initializer())
# Now all variables are initialized.
# 如果确实需要自行初始化变量，则可以运行变量的初始化器操作。例如：
session.run(my_variable.initializer)
# 可以查询哪些变量尚未初始化。例如，以下代码会打印所有尚未初始化的变量名称：
print(session.run(tf.report_uninitialized_variables()))

[]


请注意，默认情况下，tf.global_variables_initializer 不会指定变量的初始化顺序。因此，如果变量的初始值取决于另一变量的值，那么很有可能会出现错误。

最好使用 variable.initialized_value()，而非 variable：

## 使用变量
要在 TensorFlow 图中使用 tf.Variable 的值，只需将其视为普通 tf.Tensor 即可：

In [12]:
with tf.Session() as sess:
    v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
    w = tf.get_variable("w", initializer=v.initialized_value() + 1)
    sess.run(tf.global_variables_initializer())
    w = v + 1 
    print(sess.run(w))

1.0


要为变量赋值，使用 assign方法。例如，以下就是调用这些方法的方式：

In [13]:
with tf.Session() as sess:
    assignment = v.assign(1)
    sess.run(tf.global_variables_initializer())
    print(sess.run(assignment))  # or assignment.eval()

1.0
